In [14]:
import ply.lex as lex
import ply.yacc as yacc
import sys # Importar sys para manejar el atributo __file__
import os # Importar os para manejar rutas de archivo

class Analizador:
    tokens = (
        'ID',
        'NUMBER',
        'ASSIGN',
        'SEMI',
        'PLUS',
        'MINUS',
        'INT'
    )

    t_ASSIGN = r'='
    t_SEMI = r';'
    t_PLUS = r'\+'
    t_MINUS = r'-'
    t_ignore = ' \t'

    reserved = {
        'int': 'INT'
    }

    symbol_table = {}

    def t_ID(self, t):
        r'[a-zA-Z_][a-zA-Z0-9_]*'
        t.type = self.reserved.get(t.value, 'ID')
        return t

    def t_NUMBER(self, t):
        r'\d+'
        t.value = int(t.value)
        return t

    def t_newline(self, t):
        r'\n+'
        t.lexer.lineno += len(t.value)

    def t_error(self, t):
        print(f"Caracter ilegal '{t.value[0]}' en línea {t.lexer.lineno}")
        t.lexer.skip(1)

    def p_program(self, p):
        '''program : statements'''
        pass

    def p_statements(self, p):
        '''statements : statements statement
                      | statement'''
        pass


    def p_statement_declaration(self, p):
        'statement : INT ID SEMI'
        var_name = p[2]
        if var_name in self.symbol_table:
            print(f"Error semántico: variable '{var_name}' ya declarada")
        else:
            self.symbol_table[var_name] = 'int'

    def p_statement_assignment(self, p):
        'statement : ID ASSIGN expression SEMI'
        var_name = p[1]
        if var_name not in self.symbol_table:
            print(f"Error semántico: variable '{var_name}' no declarada")
        else:
            # Check if the variable is of type 'int' before assignment
            if self.symbol_table[var_name] != 'int':
                 print(f"Error semántico: variable '{var_name}' no es de tipo int")
            elif p[3] != 'int':
                print(f"Error semántico: tipo incorrecto para variable '{var_name}'. Se esperaba 'int', se obtuvo '{p[3]}'")


    def p_expression_binop(self, p):
        '''expression : expression PLUS expression
                      | expression MINUS expression'''
        if p[1] != 'int' or p[3] != 'int':
            print("Error semántico: operación solo permitida entre enteros")
            p[0] = 'int' # Return 'int' type on success
        else:
            print("Error semántico: operación solo permitida entre enteros")
            p[0] = 'error' # Return 'error' type on failure


    def p_expression_number(self, p):
        'expression : NUMBER'
        p[0] = 'int'

    def p_expression_id(self, p):
        'expression : ID'
        var_name = p[1]
        if var_name not in self.symbol_table:
            print(f"Error semántico: variable '{var_name}' no declarada")
            p[0] = 'error'
        else:
            p[0] = self.symbol_table[var_name]


    def p_error(self, p):
        if p:
            print(f"Error de sintaxis en '{p.value}' en línea {p.lineno}")
        else:
            print("Error de sintaxis inesperado (EOF)")

    def build(self):
        # Ensure a dummy file attribute for grammar rules if it's None
        # This needs to be done BEFORE calling yacc.yacc
        # Also, adding an output directory might help in interactive environments.
        outputdir = os.getcwd()
        if not hasattr(sys.modules['__main__'], '__file__'):
             sys.modules['__main__'].__file__ = 'ply_parser_temp_module'

        self.lexer = lex.lex(module=self)
        # Set write_tables to False to avoid the SyntaxError in parsetab.py
        self.parser = yacc.yacc(module=self, outputdir=outputdir, debug=True, write_tables=False)


    def analizar(self, codigo):
        self.parser.parse(codigo)

# -------------------
# USO DEL ANALIZADOR
# -------------------
codigo = """
int x;
x = 5;
y = x + 2;
"""

analizador = Analizador()
analizador.build()
analizador.analizar(codigo)

print("Tabla de símbolos:", analizador.symbol_table)

SyntaxError: '[' was never closed (parsetab.py, line 28)

In [15]:
import ply.lex as lex
import ply.yacc as yacc

class Analizador:
    # ---------------------
    # TOKENS DEL LÉXICO
    # ---------------------
    tokens = (
        'INT',
        'ID',
        'NUMERO',
        'PUNTOYCOMA',
        'ASIGNAR',
        'MAS',
    )

    # ---------------------
    # Expresiones Regulares
    # ---------------------
    t_INT          = r'int'
    t_ID           = r'[a-zA-Z_][a-zA-Z0-9_]*'
    t_NUMERO       = r'\d+'
    t_PUNTOYCOMA   = r';'
    t_ASIGNAR      = r'='
    t_MAS          = r'\+'

    t_ignore = ' \t'

    def t_newline(self, t):
        r'\n+'
        t.lexer.lineno += len(t.value)

    def t_error(self, t):
        print(f"Caracter ilegal '{t.value[0]}'")
        t.lexer.skip(1)

    # ---------------------
    # CONSTRUCTOR
    # ---------------------
    def __init__(self):
        self.lexer = None
        self.parser = None
        self.symbol_table = {}

    # ---------------------
    # REGLAS DE LA GRAMÁTICA
    # ---------------------
    def p_programa(self, p):
        '''programa : programa instruccion
                    | instruccion'''
        pass

    def p_instruccion_declaracion(self, p):
        'instruccion : INT ID PUNTOYCOMA'
        self.symbol_table[p[2]] = None

    def p_instruccion_asignacion(self, p):
        'instruccion : ID ASIGNAR expresion PUNTOYCOMA'
        if p[1] not in self.symbol_table:
            print(f"Error: variable '{p[1]}' no declarada.")
        else:
            self.symbol_table[p[1]] = p[3]

    def p_expresion_suma(self, p):
        'expresion : expresion MAS termino'
        p[0] = p[1] + p[3]

    def p_expresion_termino(self, p):
        'expresion : termino'
        p[0] = p[1]

    def p_termino_numero(self, p):
        'termino : NUMERO'
        p[0] = int(p[1])

    def p_termino_id(self, p):
        'termino : ID'
        if p[1] not in self.symbol_table or self.symbol_table[p[1]] is None:
            print(f"Error: variable '{p[1]}' no tiene valor.")
            p[0] = 0
        else:
            p[0] = self.symbol_table[p[1]]

    def p_error(self, p):
        if p:
            print(f"Error de sintaxis en '{p.value}'")
        else:
            print("Error de sintaxis al final del archivo")

    # ---------------------
    # MÉTODOS DEL ANALIZADOR
    # ---------------------
    def build(self):
        self.lexer = lex.lex(module=self)
        self.parser = yacc.yacc(module=self, write_tables=False, debug=False)

    def analizar(self, codigo):
        self.parser.parse(codigo)

# ---------------------
# USO DEL ANALIZADOR
# ---------------------
codigo = """
int x;
x = 5;
y = x + 2;
"""

analizador = Analizador()
analizador.build()
analizador.analizar(codigo)

print("Tabla de símbolos:", analizador.symbol_table)

Error de sintaxis en 'x'
Error: variable 'x' no declarada.
Error: variable 'x' no tiene valor.
Error: variable 'y' no declarada.
Tabla de símbolos: {}


In [16]:
pip install colorama


In [17]:
import ply.lex as lex
import ply.yacc as yacc
import json
from colorama import Fore, Style

class Analizador:
    tokens = (
        'INT',
        'ID',
        'NUMERO',
        'PUNTOYCOMA',
        'ASIGNAR',
        'MAS',
    )

    # ✅ Regla de inicio obligatoria
    start = 'programa'

    # ---------------------
    # Expresiones Regulares
    # ---------------------
    t_INT          = r'int'
    t_ID           = r'[a-zA-Z_][a-zA-Z0-9_]*'
    t_NUMERO       = r'\d+'
    t_PUNTOYCOMA   = r';'
    t_ASIGNAR      = r'='
    t_MAS          = r'\+'

    t_ignore = ' \t'

    def __init__(self):
        self.lexer = None
        self.parser = None
        self.symbol_table = {}
        self.errores = []

    def t_newline(self, t):
        r'\n+'
        t.lexer.lineno += len(t.value)

    def t_error(self, t):
        msg = f"Caracter ilegal '{t.value[0]}' en línea {t.lineno}"
        print(Fore.RED + msg + Style.RESET_ALL)
        self.errores.append(msg)
        t.lexer.skip(1)

    # ---------------------
    # Reglas de gramática
    # ---------------------
    def p_programa(self, p):
        '''programa : programa instruccion
                    | instruccion'''
        pass

    def p_instruccion_declaracion(self, p):
        'instruccion : INT ID PUNTOYCOMA'
        self.symbol_table[p[2]] = None

    def p_instruccion_asignacion(self, p):
        'instruccion : ID ASIGNAR expresion PUNTOYCOMA'
        if p[1] not in self.symbol_table:
            msg = f"Error: variable '{p[1]}' no declarada."
            print(Fore.RED + msg + Style.RESET_ALL)
            self.errores.append(msg)
        else:
            self.symbol_table[p[1]] = p[3]

    def p_expresion_suma(self, p):
        'expresion : expresion MAS termino'
        p[0] = p[1] + p[3]

    def p_expresion_termino(self, p):
        'expresion : termino'
        p[0] = p[1]

    def p_termino_numero(self, p):
        'termino : NUMERO'
        p[0] = int(p[1])

    def p_termino_id(self, p):
        'termino : ID'
        if p[1] not in self.symbol_table:
            msg = f"Error: variable '{p[1]}' no declarada."
            print(Fore.RED + msg + Style.RESET_ALL)
            self.errores.append(msg)
            p[0] = 0
        elif self.symbol_table[p[1]] is None:
            msg = f"Error: variable '{p[1]}' no tiene valor."
            print(Fore.YELLOW + msg + Style.RESET_ALL)
            self.errores.append(msg)
            p[0] = 0
        else:
            p[0] = self.symbol_table[p[1]]

    def p_error(self, p):
        if p:
            msg = f"Error de sintaxis en '{p.value}' (línea {p.lineno})"
        else:
            msg = "Error de sintaxis al final del archivo"
        print(Fore.MAGENTA + msg + Style.RESET_ALL)
        self.errores.append(msg)

    def build(self):
        self.lexer = lex.lex(module=self)
        self.parser = yacc.yacc(
            module=self,
            write_tables=False,
            debug=False,
            tabmodule="no_usar_parsetab"
        )

    def analizar(self, codigo):
        self.errores.clear()
        self.parser.parse(codigo)

        # Guardar errores en archivo
        with open("errores.log", "w") as f:
            for e in self.errores:
                f.write(e + "\n")

        # Guardar tabla de símbolos en JSON
        with open("tabla_simbolos.json", "w") as f:
            json.dump(self.symbol_table, f, indent=4)

        print("\n" + Fore.CYAN + "Tabla de símbolos:" + Style.RESET_ALL, self.symbol_table)
        if self.errores:
            print(Fore.LIGHTRED_EX + "Se encontraron errores. Ver 'errores.log'" + Style.RESET_ALL)
        else:
            print(Fore.GREEN + "Análisis completado sin errores." + Style.RESET_ALL)

# ---------------------
# USO DEL ANALIZADOR
# ---------------------
codigo = """
int x;
x = 5;
y = x + 2;
"""

analizador = Analizador()
analizador.build()
analizador.analizar(codigo)

Error de sintaxis en 'x' (línea 2)
Error: variable 'x' no declarada.
Error: variable 'x' no declarada.
Error: variable 'y' no declarada.

Tabla de símbolos: {}
Se encontraron errores. Ver 'errores.log'
